In [0]:
pip install azure-storage-blob

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-f71e9f07-3657-42df-8576-8eea9420d504
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
import requests
import pandas as pd
import json
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.core.pipeline.transport import HttpResponse
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, explode,to_date, date_format, count, desc, countDistinct, regexp_extract,regexp_replace
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType
import gzip
import plotly.graph_objects as go

In [0]:
spark = SparkSession.builder.getOrCreate()

storage_account_name = "pod4projectstorage"
storage_account_access_key = "2hClDrVPLGX4QBDBk8OylAkHqczIQfDja66Yl488rmj/0+vb+CAzOxL5qMe5XyM9ZupgwveVRm3N+AStriO5vg=="

container_name = "pod4-project-blob-container"
directory_path = ""

spark.conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)

df = spark.read.parquet("wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net/" + directory_path)


In [0]:
push_df = df.filter(df.type == "PushEvent")

In [0]:
for col_name in push_df.columns:
    new_col_name = col_name.replace("payload.", "")
    push_df = push_df.withColumnRenamed(col_name, new_col_name)
push_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|     type|public|          created_at|                 org|               actor|                repo|action|              before|             commits|repository_id|review|pages|    push_id|pusher_type|                 ref|size|ref_type|
+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|PushEvent|  true|2023-05-01T00:00:00Z|                null|{https://avatars....|{622742349, ihsan...|  null|ea8e2f6ddc2b0b90b...|[{{ihsannurul@hot...|    622742349|  null| null|13478051933|       null|   refs/heads/master|   1|    null|
|PushEvent|  true|2023-05-01T00:00:00Z|         

In [0]:
push_df = push_df.withColumn("commit", explode("commits")).withColumn("commit_author", col("commit.author.name")).withColumn("commit_message", col("commit.message")).withColumn("login", col("actor.login")).withColumn("url", col("commit.url")).withColumn("day_of_week", date_format(to_date(col("created_at"), "yyyy-MM-dd'T'HH:mm:ss'Z'"), "EEEE")).withColumn("domain", regexp_extract(col("url"), r".*repos/(.*)/commits/.*", 1)).drop("commits", "commit","url")

In [0]:
count_by_day = push_df.groupBy("day_of_week").count().orderBy(desc("count"))
count_by_day.show()

+-----------+--------+
|day_of_week|   count|
+-----------+--------+
|  Wednesday|17094241|
|    Tuesday|16726665|
|     Monday|16537903|
|   Thursday|14070302|
|     Friday|13783353|
|     Sunday|11023825|
|   Saturday|10884481|
+-----------+--------+



In [0]:
count_pd = count_by_day.toPandas()

fig = go.Figure(data=go.Bar(x=count_pd["day_of_week"], y=count_pd["count"]))

fig.update_layout(
    title="Number of Commits by Day of week",
    xaxis_title="Day of the Week",
    yaxis_title="Count"
)

fig.show()

In [0]:
commit_msg_by_author_counts = push_df.groupBy("commit_message").agg(countDistinct(col("commit_author")).alias("author_count")).orderBy(desc("author_count"))

In [0]:
commit_msg_by_author_counts.show()

+--------------------+------------+
|      commit_message|author_count|
+--------------------+------------+
|    Update README.md|      483535|
|Add files via upload|      424677|
|    Create README.md|      145631|
|   Update index.html|       82781|
|              update|       68786|
|                test|       38987|
|        first commit|       37281|
|        Create CNAME|       32629|
|                 fix|       30865|
|    Delete README.md|       27643|
|      Initial commit|       27084|
|             Updates|       24410|
|        Delete CNAME|       24186|
|      initial commit|       22223|
|                   .|       21493|
|              commit|       19131|
|   Delete index.html|       19128|
|              Update|       17724|
|   Create index.html|       17059|
|    Update style.css|       15654|
+--------------------+------------+
only showing top 20 rows



In [0]:
top_commit_messages = commit_msg_by_author_counts.limit(10).toPandas()
fig = go.Figure(data=go.Bar(x=top_commit_messages['commit_message'], y=top_commit_messages['author_count']))
fig.update_layout(title='Top 10 Commit Messages by Author Count', xaxis_title='Commit Message', yaxis_title='Author Count')
fig.show()

In [0]:
commit_by_domain_counts = push_df.groupBy("domain").agg(count("*").alias("commit_count")).orderBy(desc("commit_count"))

In [0]:
commit_by_domain_counts.show()

+--------------------+------------+
|              domain|commit_count|
+--------------------+------------+
|mizzled/comitigation|     4079680|
|pythoneerHiro/__e...|      659391|
|hqshit/i-hate-summer|      409384|
|   rmayr/rediffusion|      284868|
|  happyfish2024/mins|      281248|
|WolseyBankWitness...|      235772|
|bullet-dev-team/p...|      232328|
|     mo9a7i/time_now|      206438|
|   Nasfame/__garbage|      163920|
|networkoperator/d...|      161908|
|Christoffel-T/fiv...|      151895|
|jeyserHiro/__garbage|      148620|
|pythoneerHiro/__g...|      136420|
|bullet-dev-team/d...|      118041|
|DF-Master/DF-Mast...|      116782|
|Hardwaregore/gimm...|      115728|
|pythoneerHiro/__t...|      107106|
|    vpnsuperapp/fast|       99840|
|    Zebrainy/upptime|       99340|
|splashingdiamonds...|       94264|
+--------------------+------------+
only showing top 20 rows



In [0]:
top_commit_domains = commit_by_domain_counts.limit(10).toPandas()
fig = go.Figure(data=go.Bar(x=top_commit_domains['domain'], y=top_commit_domains['commit_count']))
fig.update_layout(title='Top 10 Commit domains', xaxis_title='domains', yaxis_title='Count')
fig.show()